<a href="https://colab.research.google.com/github/ashish-tikle/Pannel_PDF_QA/blob/main/QuantiphiQandA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Essential Libraries for Transformers, LangChain, Sentence-BERT, ChromaDB, Tiktoken, PyPDF, and Panel.

In [ ]:
!pip install transformers langchain langchain_community sentence-transformers chromadb tiktoken pypdf panel

In [2]:
!pip install --upgrade --quiet langchain-google-genai

In [3]:
!pip install -q -U google-generativeai

Implementing Retrieval-based Question Answering with LangChain and Google Generative AI using Transformers

In [4]:
import os
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAI
from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm
from langchain.vectorstores import Chroma
import panel as pn
import tempfile


Interactive Text Editor with Custom Styling in a Panel Dashboard

In [5]:
pn.extension('texteditor', template="bootstrap", sizing_mode='stretch_width')
pn.state.template.param.update(
    main_max_width="690px",
    header_background="#F08080",
)

Interactive Notebook for Question Answering with Advanced Settings and Google API Integration

In [6]:
file_input = pn.widgets.FileInput(width=300)

google_api_key = pn.widgets.PasswordInput(
    value="", placeholder="Enter your Google API Key here...", width=300
)

prompt = pn.widgets.TextEditor(
    value="", placeholder="Enter your questions here...", height=160, toolbar=False
)
run_button = pn.widgets.Button(name="Run!")

select_k = pn.widgets.IntSlider(
    name="Number of relevant chunks", start=1, end=5, step=1, value=2
)
select_chain_type = pn.widgets.RadioButtonGroup(
    name='Chain type',
    options=['stuff', 'map_reduce', "refine", "map_rerank"],
    value='map_reduce'
)

widgets = pn.Row(
    pn.Column(prompt, run_button, margin=5),
    pn.Card(
        "Chain type:",
        pn.Column(select_chain_type, select_k),
        title="Advanced settings"
    ), width=670
)

Semantic Question Answering with Document Retrieval using Google's Palm Embeddings and Generative AI

In [7]:
def qa(file, query, chain_type, k):
    # load document
    loader = PyPDFLoader(file)
    documents = loader.load()
    # split the documents into chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_documents(documents)
    # select which embeddings we want to use
    embeddings = GooglePalmEmbeddings()

    # create the vectorestore to use as the index
    db = Chroma.from_documents(texts, embeddings)
    # expose this index in a retriever interface
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    # create a chain to answer questions
    qa = RetrievalQA.from_chain_type(
        llm=GoogleGenerativeAI(model="models/text-bison-001"),
        chain_type=chain_type, retriever=retriever, return_source_documents=True)
    result = qa({"query": query})
    print(result['result'])
    return result

Interactive Q&A with PDFs: Store Conversations, Cache PDFs, and Display Results Dynamically.

In [8]:

convos = []  # store all panel objects in a list
os.makedirs("/.cache", exist_ok=True)  # Create the directory if it doesn't exist


def qa_result(_):
  os.environ["GOOGLE_API_KEY"] = google_api_key.value
  # save pdf file to a temp file
  if file_input.value is not None:
    file_input.save("/.cache/temp.pdf")
    prompt_text = prompt.value
    if prompt_text:
      result = qa(file="/.cache/temp.pdf", query=prompt_text, chain_type=select_chain_type.value, k=select_k.value)
      convos.extend([
                pn.Row(
                    pn.panel("\U0001F60A", width=10),
                    prompt_text,
                    width=600
                ),
                pn.Row(
                    pn.panel("\U0001F916", width=10),
                    pn.Column(
                        result["result"],
                        "Relevant source text:",
                        pn.pane.Markdown('\n--------------------------------------------------------------------\n'.join(doc.page_content for doc in result["source_documents"]))
                    )
                )
            ])
    return pn.Column(*convos, margin=15, width=575, min_height=400)

# New Section

Interactive QA Dashboard with Loading Indicator.

In [9]:
qa_interactive = pn.panel(
    pn.bind(qa_result, run_button),
    loading_indicator=True,
)

Interactive QA Output Display in a 670px Scrollable WidgetBox

In [10]:
output = pn.WidgetBox('*Output will show up here:*', qa_interactive, width=670, scroll=True)

Interactive PDF Question Answering: Upload, Authenticate, and Query with Google API Key

In [ ]:
# layout
pn.Column(
    pn.pane.Markdown("""
    ## \U0001F60A! Question Answering with your PDF file

    1) Upload a PDF.  2) Enter Google API key. 3) Type a question and click "Run"

    """),
    pn.Row(file_input,google_api_key),
    output,
    widgets

).servable()